In [24]:
import multiprocessing
cpu_cores = multiprocessing.cpu_count()
print(f"CPU cores disponibles: {cpu_cores}")

CPU cores disponibles: 8


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# === Credenciales ===
PG_URL  = 'jdbc:postgresql://localhost:5432/graphs'
PG_USER = 'spark_ingest'
PG_PASS = 'GYleZAI2pTBKJYl9W1PL'
PG_SCHEMA = 'saml_d'
PG_TABLE1 = 'accounts'
PG_TABLE2 = 'transferences'

JDBC_JAR = r"C:\spark\spark-4.0.1-bin-hadoop3\jars\postgresql-42.7.4.jar"
JDBC_BATCHSIZE = 10000
JDBC_FETCHSIZE = 10000

NEO4J_URI  = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASS = "Banco.69"
NEO4J_DDBB = "saml-d"

spark = (
    SparkSession.builder
    .appName("postgres-to-neo4j-graph")
    .config("spark.jars", JDBC_JAR)
    .config("spark.driver.extraClassPath", JDBC_JAR)
    .config("spark.executor.extraClassPath", JDBC_JAR)
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")


In [9]:
jdbc_props = {
    "user": PG_USER,
    "password": PG_PASS,
    "driver": "org.postgresql.Driver",
    "fetchsize": str(JDBC_FETCHSIZE)
}

accounts_df = (
    spark.read
         .format("jdbc")
         .option("url", PG_URL)
         .option("dbtable", f"{PG_SCHEMA}.{PG_TABLE1}")
         .options(**jdbc_props)
         .load()
)

tx_df = (
    spark.read
         .format("jdbc")
         .option("url", PG_URL)
         .option("dbtable", f"{PG_SCHEMA}.{PG_TABLE2}")
         .options(**jdbc_props)
         .load()
)

accounts_df.printSchema()
tx_df.printSchema()


root
 |-- account: long (nullable = true)
 |-- location: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- date_time: timestamp (nullable = true)
 |-- sender_account: long (nullable = true)
 |-- receiver_account: long (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_currency: string (nullable = true)
 |-- received_currency: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- is_laundering: integer (nullable = true)
 |-- laundering_type: string (nullable = true)



In [25]:
# Nodos
nodes_df = accounts_df.select(
    F.col("account").cast("long").alias("account"),
    F.col("location").alias("location")
).dropDuplicates(["account"])

# Aristas
edges_df = tx_df.select(
    F.col("id").cast("long").alias("id"),
    F.col("date_time").alias("timestamp"),
    F.col("sender_account").cast("long").alias("src"),
    F.col("receiver_account").cast("long").alias("dst"),
    F.col("amount").cast("double").alias("amount"),
    F.col("payment_currency"),
    F.col("received_currency"),
    F.col("payment_type"),
    F.col("is_laundering").cast("int"),
    F.col("laundering_type")
)

# masked ~ Bernoulli(0.2)
edges_df = edges_df.withColumn("masked", (F.rand(seed=42) < F.lit(0.2)).cast("int"))

# Opcional: particiona por destino para paralelismo estable
edges_df = edges_df.repartition(6, "dst")
nodes_df = nodes_df.repartition(2, "account")

In [10]:
from py2neo import Graph
graph = Graph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS), name=NEO4J_DDBB)
graph.run("""
CREATE CONSTRAINT account_unique IF NOT EXISTS
FOR (a:Account) REQUIRE a.account IS UNIQUE
""")
graph.run("""
CREATE CONSTRAINT tx_unique IF NOT EXISTS
FOR ()-[r:TRANSACTION]-() REQUIRE r.id IS UNIQUE
""")


(No data)

In [29]:
NEO4J_BATCHSIZE = 1000
RETRIES = 3
SLEEP = 1.0

def _get_graph():
    # Reintentos por si la primera conexión falla
    last = None
    for _ in range(RETRIES):
        try:
            return Graph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS), name=NEO4J_DDBB)
        except Exception as e:
            last = e; time.sleep(SLEEP)
    raise last

def write_nodes_partition(rows_iter):
    graph = _get_graph()
    buf = []
    run = graph.run
    def flush():
        if not buf: return
        run("""
        UNWIND $rows AS row
        MERGE (a:Account {account: row.account})
        SET a.location = row.location
        """, rows=buf)
        buf.clear()
    for row in rows_iter:
        buf.append({"account": int(row["account"]), "location": row["location"]})
        if len(buf) >= NEO4J_BATCHSIZE: flush()
    flush()

def write_edges_partition(rows_iter):
    graph = _get_graph()
    buf = []
    run = graph.run
    cypher = """
    UNWIND $rows AS row
    MERGE (s:Account {account: row.src})
    MERGE (t:Account {account: row.dst})
    MERGE (s)-[r:TX {id: row.id}]->(t)
    SET  r.timestamp = row.timestamp,
         r.amount = row.amount,
         r.payment_currency = row.payment_currency,
         r.received_currency = row.received_currency,
         r.payment_type = row.payment_type,
         r.is_laundering = row.is_laundering,
         r.laundering_type = row.laundering_type,
         r.masked = row.masked
    """
    def flush():
        if not buf: return
        run(cypher, rows=buf); buf.clear()
    for row in rows_iter:
        buf.append({
            "id": int(row["id"]),
            "src": int(row["src"]),
            "dst": int(row["dst"]),
            "timestamp": row["timestamp"],
            "amount": float(row["amount"]) if row["amount"] is not None else None,
            "payment_currency": row["payment_currency"],
            "received_currency": row["received_currency"],
            "payment_type": row["payment_type"],
            "is_laundering": int(row["is_laundering"]) if row["is_laundering"] is not None else None,
            "laundering_type": row["laundering_type"],
            "masked": int(row["masked"])
        })
        if len(buf) >= NEO4J_BATCHSIZE: flush()
    flush()

nodes_df.foreachPartition(write_nodes_partition)
edges_df.foreachPartition(write_edges_partition)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 11.0 failed 1 times, most recent failure: Lost task 1.0 in stage 11.0 (TID 20) (DESKTOP-453KJ4K executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:252)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:374)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:338)
	at org.apache.

In [ ]:
#Utilizando el bulk de PySpark
# Nodos
(nodes_df
 .write
 .format("org.neo4j.spark.DataSource")
 .mode("Overwrite")  # o "Append"
 .option("url", "bolt://localhost:7687")
 .option("authentication.type", "basic")
 .option("authentication.basic.username", NEO4J_USER)
 .option("authentication.basic.password", NEO4J_PASS)
 .option("labels", ":Account")
 .option("node.keys", "account")            # clave para MERGE
 .save()
)

# Aristas (usa id como clave de relación)
(edges_df
 .withColumnRenamed("src", "source")
 .withColumnRenamed("dst", "target")
 .write
 .format("org.neo4j.spark.DataSource")
 .mode("Overwrite")  # o "Append"
 .option("url", "bolt://localhost:7687")
 .option("authentication.type", "basic")
 .option("authentication.basic.username", NEO4J_USER)
 .option("authentication.basic.password", NEO4J_PASS)
 .option("relationship", "TX")
 .option("relationship.save.strategy", "keys")  # usa claves
 .option("relationship.keys", "id")             # clave única de la relación
 .option("relationship.source.labels", ":Account")
 .option("relationship.source.node.keys", "account:source")
 .option("relationship.target.labels", ":Account")
 .option("relationship.target.node.keys", "account:target")
 .save()
)


In [28]:
spark.range(10).repartition(2).foreachPartition(lambda it: None)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 8.0 failed 1 times, most recent failure: Lost task 1.0 in stage 8.0 (TID 18) (DESKTOP-453KJ4K executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:252)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:374)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:338)
	at org.apache.sp